# Exploratory Analysis of Diabetes Risk Factors (Solution)

Adapted from Dipanjan Sarkar et al. 2018. [Practical Machine Learning with Python](https://link.springer.com/book/10.1007/978-1-4842-3207-1).

## Overview

This module covers exploratory data analysis of diabetes risk factors using physiological measurements. We'll analyze data from the Pima Indians Diabetes Dataset to understand patterns and relationships between health metrics and diabetes outcomes.

## Learning Objectives

- Understand and analyze relationships between different diabetic risk factors
- Apply statistical analysis techniques to wine attribute data
  - Perform descriptive statistical analysis
  - Conduct inferential statistical tests like ANOVA
- Create effective visualizations to explore wine data patterns
  - Generate univariate distribution plots
  - Produce multivariate relationship plots
- Build analytical frameworks for wine type and quality prediction

### Tasks to complete

- Perform descriptive statistical analysis
- Conduct inferential statistical tests
- Generate univariate distribution visualizations
- Create multivariate relationship plots
- Analyze patterns between wine attributes and quality

## Prerequisites

- Python programming environment
- Basic understanding of statistical and machine learning concepts
- Familiarity with common ML libraries

## Get Started

- Please select kernel "conda_tensorflow2_p310" from SageMaker notebook instance.

## Problem Statement

"Can we identify key physiological risk factors for diabetes using clinical measurements?" We'll analyze relationships between health metrics and diabetes outcomes using the CRISP-DM framework.

Dataset source: [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/14/diabetes)

### Import necessary libraries

In [ ]:
# Imports the matplotlib library, specifically the 'matplotlib' module, and assigns it the alias 'mpl' for easier access.
import matplotlib as mpl
# Imports the pyplot module from matplotlib, which provides a collection of functions that make matplotlib work like MATLAB, and assigns it the alias 'plt'.
import matplotlib.pyplot as plt
# Imports the numpy library, which is fundamental for numerical computations in Python, and assigns it the alias 'np'.
import numpy as np
# Imports the pandas library, essential for data manipulation and analysis, particularly using DataFrames, and assigns it the alias 'pd'.
import pandas as pd
# Imports the 'read_csv' function specifically from the 'pandas' library to read CSV files into DataFrames.
from pandas import read_csv
# Imports the seaborn library, which is built on top of matplotlib and provides a high-level interface for creating informative statistical graphics, and assigns it the alias 'sns'.
import seaborn as sns
# Imports the 'stats' module from the scipy library, which contains functions for statistical calculations and tests.
from scipy import stats

# Sets up matplotlib to display plots directly in the output of the Jupyter notebook cells that produce plotting commands.
%matplotlib inline

# Pima Indians Diabetes Dataset

## Overview
The **Pima Indians Diabetes Dataset** is a well-known dataset used for binary classification tasks in machine learning, specifically for predicting whether a patient has diabetes based on various medical attributes. The dataset originates from the **National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK)** and focuses on female patients of **Pima Indian heritage**.

## Source
- **Dataset Repository:** [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/12/pima+indians+diabetes)
- **Original Source:** National Institute of Diabetes and Digestive and Kidney Diseases
- **Purpose:** Predicting the onset of diabetes based on diagnostic measurements.

## Dataset Description
The dataset contains **768 samples** with **8 numerical features** and **1 binary target variable** (diabetes outcome).

### **Features:**
1. **Pregnancies** – Number of times pregnant  
2. **Glucose** – Plasma glucose concentration over 2 hours in an oral glucose tolerance test  
3. **BloodPressure** – Diastolic blood pressure (mm Hg)  
4. **SkinThickness** – Triceps skinfold thickness (mm)  
5. **Insulin** – 2-Hour serum insulin (mu U/ml)  
6. **BMI** – Body mass index (weight in kg/(height in m²))  
7. **DiabetesPedigreeFunction** – Diabetes pedigree function (genetic influence)  
8. **Age** – Age of the patient (years)  
9. **Outcome** – Binary classification (1 = Diabetic, 0 = Non-Diabetic)  

## Summary Statistics
- **Total samples:** 768  
- **Diabetes positive cases (Outcome = 1):** ~35%  
- **Diabetes negative cases (Outcome = 0):** ~65%  
- **Missing values:** Some attributes contain zero values which may indicate missing data (e.g., Glucose, BloodPressure).

## Example Usage
This dataset is frequently used in **machine learning** and **statistical modeling** for:
- Logistic Regression
- Decision Trees & Random Forests
- Support Vector Machines (SVM)
- Deep Learning
- Feature Engineering and Imputation Techniques

## References
- UCI Machine Learning Repository: [Pima Indians Diabetes Dataset](https://archive.ics.uci.edu/dataset/12/pima+indians+diabetes)
- Smith, J. W., et al. "Using the ADAP learning algorithm to forecast the onset of diabetes mellitus." In Proceedings of the Annual Symposium on Computer Application in Medical Care. American Medical Informatics Association, 1988.

## Load pima-indians-diabetes dataset

In [ ]:
# Load pima-indians-diabetes with proper structure
diabetes_data = "../../Data/pima-indians-diabetes.csv"

# Column names from pima-indians-diabetes.names (class is first column)
columns = [
    'Pregnancies',
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI',
    'DiabetesPedigreeFunction',
    'Age',
    'Outcome'
]

# Load dataset (comma-separated, no header)
df = read_csv(
    diabetes_data,
    header=None,
    names=columns,
    na_values="?",
    sep=','
)

## Understand dataset features and values

In [ ]:
# Converts numerical 'Outcome' column to categorical 'outcome' column with string labels.
df['outcome'] = df['Outcome'].map({0: 'non-diabetic', 1: 'diabetic'})

# Create age groups
# Defines bins for age groups.
bins = [20, 30, 40, 50, 60, 100]
# Defines labels for age groups.
labels = ['20-29', '30-39', '40-49', '50-59', '60+']
# Creates 'age_group' column by categorizing 'Age' into defined bins with labels.
df['age_group'] = pd.cut(df['Age'], bins=bins, labels=labels)

# Prints concise summary of DataFrame, including column names, types, non-null values, and memory usage.
print(df.info())
# Displays the first 5 rows of the DataFrame to inspect the data.
df.head()

## Domain Knowledge

### Key Health Indicators

- **Pregnancies:** Number of times pregnant
- **Glucose:** Plasma glucose concentration (mg/dL)
- **BloodPressure:** Diastolic blood pressure (mm Hg)
- **SkinThickness:** Triceps skin fold thickness (mm)
- **Insulin:** 2-Hour serum insulin (mu U/ml)
- **BMI:** Body mass index (kg/m²)
- **DiabetesPedigreeFunction:** Diabetes risk genetic score
- **Age:** Years
- **Outcome:** Diabetes diagnosis (0 = Negative, 1 = Positive)

## Exploratory Data Analysis
### Descriptive Statistics

In [ ]:
# Summary statistics by diabetes outcome
# Creates a DataFrame 'diabetic' containing data for patients with Outcome=1 (diabetic).
diabetic = df[df['Outcome'] == 1]
# Creates a DataFrame 'non_diabetic' containing data for patients with Outcome=0 (non-diabetic).
non_diabetic = df[df['Outcome'] == 0]

# Defines a list of feature names 'subset_vars' for statistical analysis.
subset_vars = ['Glucose', 'BMI', 'Age', 'BloodPressure', 'Insulin']
# Calculates descriptive statistics for 'subset_vars' in 'diabetic' DataFrame and rounds to 2 decimal places.
stats_diabetic = round(diabetic[subset_vars].describe(), 2)
# Calculates descriptive statistics for 'subset_vars' in 'non_diabetic' DataFrame and rounds to 2 decimal places.
stats_non_diabetic = round(non_diabetic[subset_vars].describe(), 2)

# Concatenates 'stats_diabetic' and 'stats_non_diabetic' DataFrames side-by-side (axis=1).
pd.concat([stats_diabetic, stats_non_diabetic],
          axis=1,
          # Sets column keys for the concatenated DataFrame to 'Diabetic Patients' and 'Non-Diabetic Patients'.
          keys=['Diabetic Patients', 'Non-Diabetic Patients'])


### Inferential Statistics

#### ANOVA

A great statistical model to prove or disprove the difference in mean among subsets of data is to use
the one-way ANOVA test. ANOVA stands for “analysis of variance,” which is a nifty statistical model and can
be used to analyze statistically significant differences among means or averages of various groups. This is
basically achieved using a statistical test that helps us determine whether or not the means of several groups
are equal.

- The null hypothesis $H_0$ indicates that the group means for the various
  groups are not very different from each other based on statistical significance levels.
- The alternative
  hypotheses, $H_A$, tells us that there exists at least two group means that are statistically significantly different
  from each other.

Usually the F-statistic and the associated p-value from it is used to determine the statistical
significance. Typically a p-value less than 0.05 is taken to be a statistically significant result where we reject
the null hypothesis in favor of the original.
In our case, three data subsets or groups from the data are created based on wine quality ratings. The
mean values in the first test would be based on the wine alcohol content and the second test would be based
on the wine pH levels. Also let’s assume the null hypothesis is that the group means for low, medium, and high
quality wine is same and the alternate hypothesis would be that there is a difference (statistically significant)
between at least two group means.

In [ ]:
# Performs one-way ANOVA test to compare Glucose levels across different age groups.
F, p = stats.f_oneway(
    # Selects 'Glucose' values for age group '20-29' from DataFrame 'df'.
    df[df['age_group'] == '20-29']['Glucose'],
    # Selects 'Glucose' values for age group '30-39' from DataFrame 'df'.
    df[df['age_group'] == '30-39']['Glucose'],
    # Selects 'Glucose' values for age group '40-49' from DataFrame 'df'.
    df[df['age_group'] == '40-49']['Glucose']
)
# Prints a descriptive title for the ANOVA test being performed.
print("ANOVA for Glucose Levels Across Age Groups")
# Prints the F statistic and p-value from the ANOVA test, formatted to 2 and 4 decimal places respectively.
print(f"F Statistic: {F:.2f}\tp-value: {p:.4f}")

### Interpreting the Output:
- F-statistic = 11.35: A higher F-value suggests stronger evidence that at least one group's mean glucose level differs from the others.
- p-value = 0.0000 (< 0.05): This is extremely low, indicating that the differences in glucose levels across age groups are statistically significant at the 95% confidence level.

Since the p-value is very small (close to 0), we reject the null hypothesis (H₀), which states that all age groups have the same mean glucose levels. This means at least one age group has significantly different glucose levels compared to the others.

### Univariate Analysis

Univariate
analysis involves analyzing data such that at any instance of analysis we are only dealing with one variable or
feature. No relationships or correlations are analyzed among multiple variables. The simplest way to easily
visualize all the variables in your data is to build some histograms.

In [ ]:
# Create a figure and a set of subplots with 2 rows and 2 columns, setting the figure size to 14x10 inches.
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Create a histogram for 'Glucose' feature, colored by 'outcome', with 20 bins, placing it in the top-left subplot (axes[0,0]) and showing Kernel Density Estimate (KDE).
sns.histplot(df, x='Glucose', hue='outcome', bins=20, ax=axes[0,0], kde=True)
# Create a histogram for 'BMI' feature, colored by 'outcome', with 20 bins, placing it in the top-right subplot (axes[0,1]) and showing Kernel Density Estimate (KDE).
sns.histplot(df, x='BMI', hue='outcome', bins=20, ax=axes[0,1], kde=True)
# Create a histogram for 'Age' feature, colored by 'outcome', with 15 bins, placing it in the bottom-left subplot (axes[1,0]) and showing Kernel Density Estimate (KDE).
sns.histplot(df, x='Age', hue='outcome', bins=15, ax=axes[1,0], kde=True)
# Create a histogram for 'DiabetesPedigreeFunction' feature, colored by 'outcome', with 15 bins, placing it in the bottom-right subplot (axes[1,1]) and showing Kernel Density Estimate (KDE).
sns.histplot(df, x='DiabetesPedigreeFunction', hue='outcome', bins=15, ax=axes[1,1], kde=True)

# Adjust subplot parameters to provide reasonable spacing between subplots.
plt.tight_layout()
# Set the suptitle (overall title) for the figure to "Health Metric Distributions by Diabetes Status", adjusting vertical position for better layout.
plt.suptitle("Health Metric Distributions by Diabetes Status", y=1.02)

### Health Metric Distributions by Diabetes Status

This figure visualizes the distribution of four key health metrics (**Glucose, BMI, Age, and Diabetes Pedigree Function**) for diabetic and non-diabetic individuals. Each subplot represents the histogram along with a Kernel Density Estimation (KDE) plot.

#### **Understanding the Four Subplots:**
Each plot compares the metric distributions for:
- **Diabetic individuals** (blue bars & KDE line)
- **Non-diabetic individuals** (orange bars & KDE line)

##### **1️⃣ Glucose Distribution (Top-left)**
- **Non-diabetics:** Peak around **100 mg/dL**.
- **Diabetics:** Wider distribution, with many having glucose levels **above 120 mg/dL**.
- **Insight:** Higher glucose levels are more common in diabetics.

##### **2️⃣ BMI (Body Mass Index) Distribution (Top-right)**
- Both groups have a normal-like distribution.
- **Diabetics:** Slightly higher BMI values.
- **Non-diabetics:** Peak BMI is **25-30**, whereas diabetics have a broader range.
- **Insight:** Higher BMI is associated with diabetes risk.

##### **3️⃣ Age Distribution (Bottom-left)**
- **Non-diabetics:** Sharp peak in the **20-30 age range**.
- **Diabetics:** More evenly distributed.
- **Insight:** **Diabetes risk increases with age**.

##### **4️⃣ Diabetes Pedigree Function (DPF) Distribution (Bottom-right)**
- **Non-diabetics:** Mostly have a **low DPF (<0.5)**.
- **Diabetics:** More spread, meaning some have a strong genetic predisposition.
- **Insight:** Genetics plays a role but is not the sole determinant of diabetes.

#### **Key Takeaways:**
- Higher glucose levels, BMI, and age are associated with diabetes.
- Non-diabetics tend to be younger with lower glucose levels.
- Genetics (DPF) can contribute, but lifestyle factors (BMI, age) also play a role.


### Multivariate Analysis

Analyzing multiple feature variables and their relationships is what multivariate analysis is all about. We
would want to see if there are any interesting patterns and relationships among the physicochemical
attributes of our wine samples, which might be helpful in our modeling process in the future.

One of the best
ways to analyze features is to build a pairwise correlation plot depicting the correlation coefficient between
each pair of features in the dataset.


In [ ]:
# Selects only numeric columns from the DataFrame 'df' and stores them in 'numeric_df'.
numeric_df = df.select_dtypes(include=[np.number])
# Calculates the pairwise correlation of columns for the 'numeric_df' DataFrame.
corr = numeric_df.corr()

# Set the figure size for the heatmap to 12x8 inches.
plt.figure(figsize=(12, 8))
# Creates a heatmap to visualize the correlation matrix 'corr'.
# 'annot=True' displays correlation values on the heatmap.
# 'cmap='coolwarm'' sets the color scheme to 'coolwarm'.
# 'fmt=".2f"' formats the annotation values to two decimal places.
# 'annot_kws={'size': 10}' sets the font size of annotations.
# 'linewidths=0.5' sets the width of lines that divide heatmap cells.
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f",
            annot_kws={'size': 10}, linewidths=0.5)
# Sets the title of the heatmap plot to "Diabetes Risk Factor Correlation Matrix" and adds padding below the title.
plt.title("Diabetes Risk Factor Correlation Matrix", pad=20)
# Displays the generated heatmap plot.
plt.show()

### **Diabetes Risk Factor Correlation Matrix**

#### **Understanding the Heatmap**
This heatmap represents the **correlation coefficients** between different numeric variables in the dataset. The correlation values range from **-1 to 1**:
- **1.0 (Red):** Strong positive correlation (variables increase together)
- **0.0 (White):** No correlation
- **-1.0 (Blue):** Strong negative correlation (one increases, the other decreases)

#### **Key Observations**
- **Glucose & Outcome (0.47):** Moderate positive correlation. Higher glucose levels are associated with diabetes.
- **BMI & Outcome (0.29):** Higher BMI tends to correlate with diabetes risk.
- **Age & Outcome (0.24):** Older individuals are slightly more likely to be diabetic.
- **Pregnancies & Age (0.54):** Older individuals tend to have more pregnancies.
- **SkinThickness & Insulin (0.44):** These variables are related, likely due to their link with body fat and metabolic function.
- **Weak Correlations:** Blood Pressure, Skin Thickness, and Diabetes Pedigree Function have lower correlations with diabetes outcome.

#### **Insights**
- Glucose is the strongest predictor of diabetes in this dataset.
- BMI and Age also contribute but to a lesser extent.
- Other factors like Blood Pressure and Skin Thickness show weak relationships with diabetes.

This analysis helps identify key risk factors and their relationships, useful for feature selection in predictive modeling.


### Pairwise relationships of Diabetes Risk Factor

In [ ]:
# Set the figure size for the pairplot to 12x8 inches.
plt.figure(figsize=(12,8))
# Create a pairplot using seaborn to visualize pairwise relationships between 'Glucose', 'BMI', 'Age', and 'Insulin' features, colored by 'outcome', with transparency alpha=0.6.
sns.pairplot(df, vars=['Glucose', 'BMI', 'Age', 'Insulin'],
             hue='outcome', plot_kws={'alpha':0.6})
# Set the suptitle (overall title) for the pairplot to "Diabetes Risk Factor Relationships", adjusting vertical position for better layout.
plt.suptitle("Diabetes Risk Factor Relationships", y=1.02)

### **Diabetes Risk Factor Relationships**

#### **Understanding the Pairplot**
This pairplot visualizes **pairwise relationships** between key health metrics (`Glucose`, `BMI`, `Age`, and `Insulin`) while distinguishing between **diabetic** and **non-diabetic** individuals.

- **Diagonal Plots (Kernel Density Estimation - KDE)**  
  - These show the distribution of each variable.  
  - **Glucose:** Higher glucose levels are more frequent in diabetics.  
  - **BMI:** The distribution is similar, but diabetics tend to have a slightly higher BMI.  
  - **Age:** Diabetics are more evenly spread across different ages, while non-diabetics cluster at younger ages.  
  - **Insulin:** Skewed distribution with many low values.

- **Scatter Plots (Off-Diagonal)**
  - Each point represents an individual, color-coded by diabetes status.  
  - **Glucose vs. Insulin:** Some correlation—higher glucose is often paired with higher insulin.  
  - **Age vs. Glucose:** Weak correlation; diabetes occurs at various ages, but older individuals are at higher risk.  
  - **BMI vs. Insulin:** Some clustering, but no strong trend.  

#### **Insights**
- **Glucose is the strongest distinguishing factor** between diabetic and non-diabetic groups.  
- **Insulin values show extreme variability**, with some very high outliers.  
- **Age and BMI alone are not clear separators**, but they contribute to diabetes risk.  

This visualization helps explore feature relationships and identify patterns useful for machine learning models.


## Conclusion

Through this analysis, we learned:

- How to effectively analyze relationships between wine properties and quality
- Statistical techniques for comparing wine attribute distributions
- Methods for visualizing multi-dimensional wine data relationships
- Patterns distinguishing high vs low quality wines
- Differences between red and white wine characteristics

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.